In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from astropy.timeseries import LombScargle
import os, copy
from tqdm import tqdm

In [ ]:
def readLightCurves(inFilePath):
    """ 
    INPUT  : file path 
    OUTPUT : dictionnary t, m, e (t = time, m = magnitude, e = error) 
    """
    with open(inFilePath, 'r') as Fi:
        t, m, e = np.array([np.array([float(_) for _ in line.rstrip("\n").split()]) for line in Fi]).T
    
    return {"t" : t, "m" : m, "e" : e}  # LC = Light Curve

In [ ]:
def loadStarsInventory(source):
    """
    INPUT  - source (str) : LMC or SMC
    OUTPUT - dictionnary containing two keys. Star ID and the type corresponding (Mira, SRV, OSARG)
    """    
    starsInventory = {"ID" : [], "Type" : []}

    if source == "LMC" or source == "SMC":
        inFilePath = "{0}/Data/Inventory_{1}.txt".format(os.getcwd(), source)
    else:
        return starsInventory

    with open(inFilePath, "r") as inFile:
        content = inFile.read()
        content = content.split("\n")
        for i in range(len(content)):
            starsInventory["ID"].append(content[i][0:18])
            starsInventory["Type"].append(content[i][37:42].strip())
    return starsInventory

In [ ]:
def getStarType(starID, starsInventory):
    return starsInventory["Type"][starsInventory["ID"].index(starID)]

In [ ]:
def getStarID(inFilePath):
    """ 
    INPUT  : file path
    OUTPUT : star ID (OGLE-LMC-LPV-#####)
    """
    return inFilePath[len(str(os.getcwd()) + "/Data/XXX_phot/I_X/"):len(str(os.getcwd()) + "/Data/XXX_phot/I_X")+19]

In [ ]:
def getFilePath(starID, basePath="{}/Data/".format(os.getcwd())):
    x = starID.split("-")[-1][0]
    source = starID.split("-")[1]
    return basePath + source + "_phot/" + "I_" + x + "/" + starID + ".dat"

In [ ]:
def writeResultsInFile(outFilePath, data):
    """
    Writes data in txt file in the format :
    ID mag0 P1 P2 P3 A1 A2 A3
    """
    with open(outFilePath, "w") as outFile:
        outFile.write("#ID mag0 P1 P2 P3 A1 A2 A3\n")
        for i in range(len(data)):
            line = ""
            line += data[i]["ID"] + " "
            line += "{:.3f}".format(data[i]["mag0"]) + " "
            line += " ".join(["{:.2f}".format(_) for _ in data[i]["P"]][0:3]) + " "
            line += " ".join(["{:.2f}".format(2*abs(_)) for _ in data[i]["A"]][0:3]) + " "
            line += "\n"
            outFile.write(line)

In [ ]:
def fPeriodModel(t, mag0, A, P0, phi):
    return mag0 + A * np.cos((2*np.pi*t/P0) + phi)

In [ ]:
def analyseLightCurve(inFilePath, nPer=1, plots=False, save=False):
    global inventory

    output = {"ID" : "", "mag0" : 0, "P" : [], "A" : [], "phi" : [] }

    # Getting data from file - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
    # LC0, LSP0 : Variables which won't be modified. 
    #             This allow us to keep the initial data
    #             (Will be usefull at the end of the function to draw the plots)
    # LCi, LSPi : Variables which will be modified in the loop.
    
    LC0 = readLightCurves(inFilePath)
    starID = getStarID(inFilePath)
    
    output["ID"] = starID
    
    LSP0 = LombScargle(LC0["t"], LC0["m"], LC0["e"])
    
    LSPi = copy.deepcopy(LSP0)
    LCi = copy.deepcopy(LC0)
    
    # Total interval of time (days)
    DeltaT = np.nanmax(LC0["t"]) - np.nanmin(LC0["t"])
    
    model = {"t" : LCi["t"]*1}
    
    plotLSP = {"Periods" : [], "Powers" : []}
    
    for i in range(nPer):
        
        frequency, power = LSPi.autopower(minimum_frequency = 1/DeltaT,
                                          maximum_frequency = 0.4,
                                          samples_per_peak  = 50)
        periods = 1/frequency
        
        plotLSP["Periods"].append(periods)
        plotLSP["Powers"].append(power / np.nanmax(power))
        
        pks = signal.find_peaks(power)[0]
        # pksP : Periods corresponding to the peaks (starting from the highest peak)
        pksP = np.array([p for k, p in sorted(zip(power[pks], periods[pks]))])[::-1] 
        
        output["P"].append(pksP[0])
        
        mpar = LSPi.model_parameters(1/output["P"][i])
        
        if i == 0:
            mag0 = mpar[0] + LSP0.offset()
            output["mag0"] = mag0
            
            model["m"] = np.ones(len(model["t"])) * mag0
            LCi["m"] -= mag0
        
        output["A"].append(np.sign(mpar[2]) * np.sqrt(mpar[1]**2 + mpar[2]**2))
        output["phi"].append(np.arctan(-mpar[1]/mpar[2]))
        
        # We remove every period before the i-th period
        
        model["m"] += fPeriodModel(model["t"], 0, output["A"][i], output["P"][i], output["phi"][i])
        LCi["m"] -= fPeriodModel(model["t"], 0, output["A"][i], output["P"][i], output["phi"][i])
        
        # Applying Lomb Scargle algorithm on the i-th LC
        LSPi = LombScargle(LCi["t"], LCi["m"])
        
    # Plotting the data
    if plots:
        
        fontSize = 18 # Font size for the graphs (title, axis,...)
        
        fig = plt.figure(figsize=(6, 8), dpi=300)
        ax1 = fig.add_subplot(211)
        ax2 = fig.add_subplot(212)
        
        # Plotting original data (light curve)
        ax1.scatter(LC0["t"], LC0["m"], c="k", s=2, marker="o", zorder=15)

        xlim = ax1.get_xlim()
    
        plotModel = {"t" : np.linspace(*xlim, 1001)}
        plotModel["m"] = np.ones(len(plotModel["t"])) * mag0
        
        for i in range(nPer):
            plotModel["m"] += fPeriodModel(plotModel["t"], 0, output["A"][i], output["P"][i], output["phi"][i])
        
        ax1.plot(plotModel["t"], plotModel["m"], lw=1, c="r", ls="-", alpha=0.5, zorder=16)
        ax1.set_xlim(xlim)
        
        
        ax1.invert_yaxis()

        ax1.set_title("I-band magnitude vs time\nStar ID : " + starID, fontsize=fontSize)
        ax1.set_ylabel("I [mag]", fontsize=fontSize)
        ax1.set_xlabel("Time [days]", fontsize=fontSize)
    
        ax1.errorbar(LC0["t"], LC0["m"], yerr=LC0["e"], lw=0, ecolor="k", elinewidth=0.5, capsize=1, zorder=10) 
                       
        [ax2.plot(plotLSP["Periods"][i], plotLSP["Powers"][i]+i, c="rgbcmy"[i], lw=0.5) for i in range(nPer)]
        
        ax2.set_xscale("log")
        ax2.set_title("Periodograms", fontsize=fontSize)
        [ax2.axvline(output["P"][i], c="rgbcmy"[i], lw=0.5, zorder=-10) for i in range(nPer)]
        ax2.set_xlabel("Periods [log(days)]", fontsize=fontSize)
        ax2.set_ylabel("Power [nu]", fontsize=fontSize)
        
        plt.subplots_adjust(bottom=0.15)
        fig.tight_layout(h_pad=2.0)
        
        if save:
            outFilePath = "{0}/Output/LCs/{1}.png".format(os.getcwd(), starID)
            fig.savefig(outFilePath, bbox_inches = 'tight', facecolor = 'w')
        
    return output

In [ ]:
source = "LMC"

inventory = loadStarsInventory(source)
LEN = len(inventory["ID"])

numMax = {"Mira" : 1, "SRV" : 0, "OSARG" : 0}
IDs = {"Mira" : [], "SRV" : [], "OSARG" :[]}

for i in range(LEN):   
    
    if all([len(IDs[k]) >= numMax[k] for k in IDs]):
        break
        
    vt = inventory["Type"][i] # Variability type
    
    if len(IDs[vt]) < numMax[vt]:
        IDs[vt].append(inventory["ID"][i])

In [ ]:
allStarsData = []

for k in IDs:
    for ID in tqdm(IDs[k]):
        allStarsData.append(analyseLightCurve(getFilePath(ID), 3, True))
#outFilePath = os.getcwd() + "/Output/RESULTS_SMC.txt"
#writeResultsInFile(outFilePath, allStarsData)